In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50,InceptionResNetV2
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
base_model = InceptionResNetV2(weights = 'imagenet',include_top=False,input_shape=(224,224,3))
#base_model.summary()

In [3]:
model = Sequential()
model.add(base_model)
#x = base_model.output
# x = Flatten()(x)
# x = Dense(256,activation='relu')(x)
# predictions = Dense(2, activation='softmax')(x)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(128,activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [4]:
#model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers[:500]: 
  layer.trainable = False
for layer in base_model.layers[500:]: 
  layer.trainable = True
# for layer in base_model.layers: 
#    layer.trainable = False
model.compile(optimizer=optimizers.SGD(lr=0.001), loss='categorical_crossentropy'
              , metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 5, 5, 1536)        54336736  
_________________________________________________________________
flatten_1 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               9830656   
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
Total params: 64,168,420
Trainable params: 43,870,020
Non-trainable params: 20,298,400
_________________________________________________________________


In [5]:
model.load_weights("newrust3.h5")

In [6]:
model.save('kerasmodel3.pkl')

In [9]:
%%writefile script.py

import numpy as np
import os
import sys
import keras as K
import json
import base64
from io import BytesIO
from PIL import Image
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input

def init():
    
    global model  

    print("Executing init() method...")
    print("Python version: " + str(sys.version) + ", keras version: " + K.__version__)
    # Load the model 
    model = K.models.load_model('azureml-models/kerasmodel3/1/kerasmodel3.pkl')
    #model = K.models.load_model('kerasmodel3.pkl')
    
    return

def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict_classes(x)
    y = model.predict(x)[0]
    y = np.expand_dims(y, axis=-1)
    print(y)
    b = []
    a = y[0]
    b.append(np.float(a))
    a = y[1]
    b.append(np.float(a))
    a = y[2]
    b.append(np.float(a))
    a = y[3]
    b.append(np.float(a))
    b.append(preds[0])
    return b

def run(inputString):
    
    responses = []
    base64Dict = json.loads(inputString)

    for k, v in base64Dict.items():
        img_file_name, base64Img = k, v
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)
    #imageData = Image.open(img_buffer).convert("RGB")


#     # Evaluate the model using the input data
#     img = ImageOps.fit(imageData, (32, 32), Image.ANTIALIAS)
#     img_conv = np.array(img) # shape: (32, 32, 3)
#     # Scale pixel intensity
#     x_test = img_conv / 255.0
#     # Reshape
#     x_test = np.moveaxis(x_test, -1, 0)
#     x_test = np.expand_dims(x_test, 0)  # shape (1, 3, 32, 32)

#     y_pred = model.predict(x_test)
#     y_pred = np.argmax(y_pred, axis=-1)
#     # print(y_pred)
#     data = json.loads(inputString)['data']
    #img_input = base64.decodestring(json.dumps(inputString)[0])
    img = image.load_img(img_buffer,target_size=(224, 224))
    #img = json.loads(inputString)['data']
    preds = predict(model, img)
    LABELS = ["Category A","Category B","Category C","Category D"]
    resp = LABELS[preds[-1]]
    responses.append(preds[0])
    responses.append(preds[1])
    responses.append(preds[2])
    responses.append(preds[3])
    responses.append(resp)
    return json.dumps(responses)
    
  
if __name__ == "__main__":
    init()
    # input data
    img_path = '00000668.JPG'
    #input_data = "{\"data\": [" + str(list(img_path)) + "]}"
    encoded = None
    with open(img_path, 'rb') as file:
        encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    resp = run(body)
    print(resp)

Overwriting script.py


In [10]:
!python script.py

Using TensorFlow backend.
Executing init() method...
Python version: 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0], keras version: 2.1.5
2019-01-15 20:09:29.238418: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-01-15 20:09:29.266077: E tensorflow/stream_executor/cuda/cuda_driver.cc:300] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2019-01-15 20:09:29.266151: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:150] kernel driver does not appear to be running on this host (DemoVM): /proc/driver/nvidia/version does not exist
[[0.41764382]
 [0.07987227]
 [0.12280396]
 [0.37968   ]]
[0.41764381527900696, 0.079872265458107, 0.12280396372079849, 0.37968000769615173, "Category A"]


In [11]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.2


In [12]:
ws = Workspace(resource_group='DemoGroup',workspace_name='demospace',subscription_id='5959c85b-cacb-4c20-b1fb-e47c40ed5aff')

In [13]:
from azureml.core.model import Model

In [14]:
model_name = "kerasmodel3"
model_path = "./kerasmodel3.pkl"

model = Model.register(
    model_path=model_path,
    model_name=model_name,
    tags={"data": "rust", "model": "classification"},
    description="Classifying rust and non rust images",
    workspace=ws
)

Registering model kerasmodel3


In [15]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2, 
    memory_gb=2, 
    tags={"data": "rust",  "method" : "keras"}, 
    description='Predict rust with keras'
)

In [16]:
%%time
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(
    execution_script="script.py", 
    runtime="python", 
    conda_file="myenv.yml"
)

image = ContainerImage.create(
    name = "keras-rust-img3",
    models = [model],
    image_config = image_config,
    workspace = ws
)

image.wait_for_creation(show_output = True)

Creating image
Running..............................................................
SucceededImage creation operation finished for image keras-rust-img3:1, operation "Succeeded"
CPU times: user 1.77 s, sys: 89.8 ms, total: 1.86 s
Wall time: 5min 31s


In [17]:
print(image.image_location)

demospace3143890589.azurecr.io/keras-rust-img3:1


In [18]:
%%time
from azureml.core.webservice import Webservice

aci_service = Webservice.deploy_from_image(
    workspace = ws, 
    name = 'keras-rust-aci-svc3',
    image = image,
    deployment_config = aciconfig
)

aci_service.wait_for_deployment(show_output=True)

Creating service
Running..................................
FailedACI service creation operation finished, operation "Failed"
Service creation polling reached terminal state, unexpected error response:
{'code': 'AciDeploymentFailed', 'message': 'Aci Deployment failed', 'details': [{'code': 'CrashLoopBackOff', 'message': "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance keras-rust-aci-svc3.\nYou can also try to run image demospace3143890589.azurecr.io/keras-rust-img3:1 locally. Please refer to http://aka.ms/debugimage for more information."}]}
CPU times: user 970 ms, sys: 44.7 ms, total: 1.02 s
Wall time: 3min 20s


In [19]:
print(aci_service.scoring_uri)

http://52.236.144.199:80/score
